In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swmhau_network_functions import (
    swmhau_network_hyperparameter_search,
)

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


# swmhau Network

In [7]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = True

In [8]:
num_epochs = 100
dimensions = [15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6)]
num_layers = [1]
ffn_hidden_dim_sizes = [[256, 256], [512, 512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [9]:
kwargs = {
    "num_epochs": num_epochs,
    "df": df_rumours,
    "id_column": "timeline_id",
    "label_column": "label",
    "embeddings": sbert_embeddings,
    "y_data": y_data,
    "output_dim": output_dim,
    "dimensions": dimensions,
    "log_signature": True,
    "pooling": "signature",
    "swmhau_parameters": swmhau_parameters,
    "num_layers": num_layers,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "split_ids": split_ids,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# w=5

In [10]:
size = 5

## umap

In [11]:
(
    swmhau_network_umap_kfold_5,
    best_swmhau_network_umap_kfold_5,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_umap_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_umap_focal_2_5_kfold_best_model.csv


In [12]:
swmhau_network_umap_kfold_5.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_3053319/2619527520.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_5.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.690214   
                                                                                      0.0005         0.689966   
                                                                                      0.0010         0.694812   
                                                                         0.2          0.0001         0.695620   
                                                                                      0.0005         0.690960   
                                                                                      0.0010         0.695123   
                                                          (512, 512)     0.1          0.0001         0.690339   
                                                                                      0.0005         0.697235   
                                                                                      0.0010         0.690774   
                                                                         0.2          0.0001         0.692451   
                                                                                      0.0005         0.698913   
                                                                                      0.0010         0.691892   
           12              3         10        1          (256, 256)     0.1          0.0001         0.679901   
                                                                                      0.0005         0.690649   
                                                                                      0.0010         0.699410   
                                                                         0.2          0.0001         0.686052   
                                                                                      0.0005         0.692886   
                                                                                      0.0010         0.691333   
                                                          (512, 512)     0.1          0.0001         0.685430   
                                                                                      0.0005         0.694688   
                                                                                      0.0010         0.695433   
                                                                         0.2          0.0001         0.680149   
                                                                                      0.0005         0.694501   
                                                                                      0.0010         0.700652   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.664677   
                                                                                      0.0005         0.666264   
                                                                                      0.0010         0.669367   
                                                                         0.2          0.0001         0.668542   
                                                                                      0.0005         0.668148   
                                                                                      0.0010         0.670389   
                                                          (512, 512)     0.1          0.0001         0.665661   
                                                                                      0.0005         0.672638   
                                

In [13]:
best_swmhau_network_umap_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.688910,0.666456,"[0.752996892111884, 0.5799144223508684]",0.663919,"[0.7782196390333436, 0.549618320610687]",0.671552,"[0.7293577981651376, 0.6137453383058071]",None,0.737970,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.694315,0.667903,"[0.761558592614132, 0.5742471443406023]",0.666283,"[0.772566371681416, 0.56]",0.670049,"[0.7508600917431193, 0.5892381459776239]",None,0.739048,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.694129,0.666351,"[0.7626211485606827, 0.5700812156143569]",0.665230,"[0.7696350364963503, 0.5608247422680412]",0.667691,"[0.7557339449541285, 0.5796483750665956]",None,0.738644,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [14]:
best_swmhau_network_umap_kfold_5[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,8,4,6,1,"(512, 512)",0.2,0.0001
0,15,8,4,6,1,"(512, 512)",0.2,0.0001
0,15,8,4,6,1,"(512, 512)",0.2,0.0001


In [15]:
best_swmhau_network_umap_kfold_5["f1"].mean()

0.6669032359320877

In [16]:
best_swmhau_network_umap_kfold_5["precision"].mean()

0.6651440183483064

In [17]:
best_swmhau_network_umap_kfold_5["recall"].mean()

0.6697639490354019

In [18]:
np.stack(best_swmhau_network_umap_kfold_5["f1_scores"]).mean(axis=0)

array([0.75905888, 0.57474759])

In [19]:
np.stack(best_swmhau_network_umap_kfold_5["precision_scores"]).mean(axis=0)

array([0.77347368, 0.55681435])

In [20]:
np.stack(best_swmhau_network_umap_kfold_5["recall_scores"]).mean(axis=0)

array([0.74531728, 0.59421062])

# w=11

In [21]:
size = 11

## umap

In [22]:
(
    swmhau_network_umap_kfold_11,
    best_swmhau_network_umap_kfold_11,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_umap_focal_2_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_umap_focal_2_11_kfold_best_model.csv


In [23]:
swmhau_network_umap_kfold_11.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_3053319/404807304.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_11.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.686921   
                                                                                      0.0005         0.696303   
                                                                                      0.0010         0.697422   
                                                                         0.2          0.0001         0.687170   
                                                                                      0.0005         0.693135   
                                                                                      0.0010         0.688723   
                                                          (512, 512)     0.1          0.0001         0.684001   
                                                                                      0.0005         0.699223   
                                                                                      0.0010         0.695123   
                                                                         0.2          0.0001         0.685989   
                                                                                      0.0005         0.688040   
                                                                                      0.0010         0.694936   
           12              3         10        1          (256, 256)     0.1          0.0001         0.677540   
                                                                                      0.0005         0.686859   
                                                                                      0.0010         0.691457   
                                                                         0.2          0.0001         0.679963   
                                                                                      0.0005         0.686176   
                                                                                      0.0010         0.699845   
                                                          (512, 512)     0.1          0.0001         0.682572   
                                                                                      0.0005         0.691395   
                                                                                      0.0010         0.695744   
                                                                         0.2          0.0001         0.682759   
                                                                                      0.0005         0.700031   
                                                                                      0.0010         0.700093   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.662145   
                                                                                      0.0005         0.668994   
                                                                                      0.0010         0.674823   
                                                                         0.2          0.0001         0.664398   
                                                                                      0.0005         0.669224   
                                                                                      0.0010         0.666380   
                                                          (512, 512)     0.1          0.0001         0.661643   
                                                                                      0.0005         0.670573   
                                

In [24]:
best_swmhau_network_umap_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.678658,0.652151,"[0.7481741162722757, 0.5561277033985582]",0.650386,"[0.762656343061346, 0.5381165919282511]",0.654809,"[0.7342316513761468, 0.5753862546616942]",None,0.778272,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.676980,0.657028,"[0.7397507133203185, 0.5743060673053304]",0.654772,"[0.7767265846736046, 0.53281677301732]",0.664469,"[0.7061353211009175, 0.6228023441662227]",None,0.784742,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.676980,0.653737,"[0.7434492968171724, 0.5640251572327043]",0.651457,"[0.768595041322314, 0.5343183984747378]",0.658563,"[0.7198967889908257, 0.597229621736814]",None,0.782046,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [25]:
best_swmhau_network_umap_kfold_11[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001


In [26]:
best_swmhau_network_umap_kfold_11["f1"].mean()

0.6543055090577267

In [27]:
best_swmhau_network_umap_kfold_11["precision"].mean()

0.6522049554129289

In [28]:
best_swmhau_network_umap_kfold_11["recall"].mean()

0.6592803303387701

In [29]:
np.stack(best_swmhau_network_umap_kfold_11["f1_scores"]).mean(axis=0)

array([0.74379138, 0.56481964])

In [30]:
np.stack(best_swmhau_network_umap_kfold_11["precision_scores"]).mean(axis=0)

array([0.76932599, 0.53508392])

In [31]:
np.stack(best_swmhau_network_umap_kfold_11["recall_scores"]).mean(axis=0)

array([0.72008792, 0.59847274])

# w=20

In [10]:
size = 20

## umap

In [11]:
(
    swmhau_network_umap_kfold_20,
    best_swmhau_network_umap_kfold_20,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_umap_focal_2_20_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_umap_focal_2_20_kfold_best_model.csv


In [12]:
swmhau_network_umap_kfold_20.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_1052117/1154838213.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_20.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.684126   
                                                                                      0.0005         0.691892   
                                                                                      0.0010         0.692575   
                                                                         0.2          0.0001         0.687605   
                                                                                      0.0005         0.694004   
                                                                                      0.0010         0.696179   
                                                          (512, 512)     0.1          0.0001         0.685989   
                                                                                      0.0005         0.696490   
                                                                                      0.0010         0.695806   
                                                                         0.2          0.0001         0.681143   
                                                                                      0.0005         0.693010   
                                                                                      0.0010         0.684747   
           12              3         10        1          (256, 256)     0.1          0.0001         0.679341   
                                                                                      0.0005         0.688910   
                                                                                      0.0010         0.693880   
                                                                         0.2          0.0001         0.673874   
                                                                                      0.0005         0.680149   
                                                                                      0.0010         0.693135   
                                                          (512, 512)     0.1          0.0001         0.684188   
                                                                                      0.0005         0.684436   
                                                                                      0.0010         0.697111   
                                                                         0.2          0.0001         0.672445   
                                                                                      0.0005         0.689158   
                                                                                      0.0010         0.695433   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.659937   
                                                                                      0.0005         0.670415   
                                                                                      0.0010         0.668300   
                                                                         0.2          0.0001         0.667791   
                                                                                      0.0005         0.671456   
                                                                                      0.0010         0.671394   
                                                          (512, 512)     0.1          0.0001         0.661883   
                                                                                      0.0005         0.672663   
                                

In [13]:
best_swmhau_network_umap_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.661696,0.634317,"[0.734377286696912, 0.5342571208622017]",0.632711,"[0.7500747384155456, 0.5153465346534654]",0.636966,"[0.7193233944954128, 0.5546084176877997]",None,0.869255,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.680522,0.656052,"[0.7477928193054738, 0.5643111337061514]",0.653879,"[0.7681378476420798, 0.5396208070004861]",0.659933,"[0.7284977064220184, 0.5913692061800746]",None,0.867233,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.675116,0.655572,"[0.737618545837724, 0.5735258135551751]",0.653431,"[0.7765451664025357, 0.530316742081448]",0.663404,"[0.7024082568807339, 0.6244006393180608]",None,0.861167,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [14]:
best_swmhau_network_umap_kfold_20[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(512, 512)",0.2,0.0001
0,15,12,3,10,1,"(512, 512)",0.2,0.0001
0,15,12,3,10,1,"(512, 512)",0.2,0.0001


In [15]:
best_swmhau_network_umap_kfold_20["f1"].mean()

0.6486471199939396

In [16]:
best_swmhau_network_umap_kfold_20["precision"].mean()

0.6466736393659267

In [17]:
best_swmhau_network_umap_kfold_20["recall"].mean()

0.65343460349735

In [18]:
np.stack(best_swmhau_network_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.73992955, 0.55736469])

In [19]:
np.stack(best_swmhau_network_umap_kfold_20["precision_scores"]).mean(axis=0)

array([0.76491925, 0.52842803])

In [20]:
np.stack(best_swmhau_network_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.71674312, 0.59012609])

# w=35

In [21]:
size = 35

## umap

In [ ]:
(
    swmhau_network_umap_kfold_35,
    best_swmhau_network_umap_kfold_35,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
swmhau_network_umap_kfold_35.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_35

In [ ]:
best_swmhau_network_umap_kfold_35[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_swmhau_network_umap_kfold_35["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_35["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_35["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_35["recall_scores"]).mean(axis=0)

# w=80

In [ ]:
size = 80

## umap

In [ ]:
(
    swmhau_network_umap_kfold_80,
    best_swmhau_network_umap_kfold_80,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_umap_kfold_80.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_80

In [ ]:
best_swmhau_network_umap_kfold_80[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_swmhau_network_umap_kfold_80["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_80["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_80["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["recall_scores"]).mean(axis=0)